The goal of this project is to perform image compression using SVD method.

If $\mathrm{Img}$ is an $m$-by-$n$ matrix representing a grayscale image, then a rank-{r_0} approximation is given by
\begin{equation}
 \mathrm{Img}_{r_0} = U_{r_0}\Sigma_{r_0}V_{r_0}^T
\end{equation}
where $U_{r_0}$ is a matrix of size $m$-by-$r_0$ containing the first $r_0$ left singular vectors of $\mathrm{Img}$, $V_{r_0}$ is a matrix of size $n$-by-$r_0$ containing the first $r_0$ right singular vectors of $\mathrm{Img}$, and $\Sigma_{r_0}$ is a diagonal matrix that contains the first $r_0$ non-zero singular values.
This approximation gives a compression ratio
\begin{equation}
    \mathrm{CR}(r_0)= \frac{r_0\times(1+n+m)}{\text{size of }\mathrm{Img}}
\end{equation}

For this project I will implement these functions:


1.   `get_compression_ratio`: Returns the compression ratio for a given rank.
2.   `get_threshold_rank`: Returns the rank-$r_0$ such that $\sigma_{r_0}/\sigma_0\leq \mathrm{thrsld}$.
3.   `compress_img`: Returns the matrices $U_r$, $V_r$, and an array of singular values $S_r=[\sigma_0,\sigma_1,\cdots,\sigma_{r-1}]$, for a given rank $r$.
4.    `rebuild_img`: Returns $\mathrm{Img}_r$ given $U_r$, $V_r$, and $[\sigma_0,\sigma_1,\cdots,\sigma_{r-1}]$.
5.    `get_compression_error`: Returns the normalized 2-norm of the reconstrution error $||\mathrm{Img}-\mathrm{Img}_r||/||\mathrm{Img}||$

In [1]:
# --------------------------------------------------
# Imports section
# --------------------------------------------------
import numpy as np
import requests
from PIL import Image
from io import BytesIO

1) Implemented a function that computes the compression ratio
\begin{equation}
    \mathrm{CR}(r_0)= \frac{r_0\times(1+n+m)}{\text{size of }\mathrm{Img}}
\end{equation}

In [2]:
# ---------------------------------
# Function: get_compression_ratio
# ---------------------------------
'''
:inputs  : img            - Input grayscale image (MxN numpy array)
:inputs  : rank           - rank approxiamtion value (integer)
:returns : CR             - Compression Ratio (float)
'''

def get_compression_ratio (img, rank):

    W,H=img.shape

    Image_size=W*H
    CR=(rank*(1+W+H))/(Image_size)

    return (CR)

2) Implemented a function that finds that returns the rank-$r_0$ such that $\sigma_{r_0}/\sigma_0\leq \mathrm{thrsld}$.

In [3]:
# ---------------------------------
# Function: get_threshold_rank
# ---------------------------------
'''
:inputs  : img            - Input grayscale image (MxN numpy array)
:inputs  : thrsld         - threshold (float)
:returns : rank           - rank that meets the threshold (integer)
'''

def get_threshold_rank (img,thrsld):


    U, Sigma , V_T = np.linalg.svd(img, full_matrices=False)
    n=Sigma.shape[0]
    k=0;
    for i in range(n):
      if Sigma[i]/Sigma[0]<=thrsld:
        return i
    for j in range(n):
      if Sigma[j]/Sigma[0]>thrsld:
        k=k+1;
    rank = k;
    return rank









3) Implemented a function that builds that returns the matrices $U_r$, $V_r$, and an array of singular values $S_r=[\sigma_0,\sigma_1,\cdots,\sigma_{r-1}]$, for a given rank $r$.

In [4]:
# ---------------------------------
# Function: compress_img
# ---------------------------------
'''
:inputs  : img            - Input grayscale image (MxN numpy array)
:inputs  : rank           - rank that meets the threshold (integer)
:returns : U              - left singular vectors (Mxr 2D-numpy array)
:returns : V              - right singular vectors (Mxr 2D-numpy array)
:returns : S              - singular values (numpy array of size r)
'''

def compress_img (img,rank):

    U, Sigma , V_T = np.linalg.svd(img)

    U=U[:,:rank]

    V=V_T[:rank,:]
    V=V.T
    S=Sigma[:rank]

    return (U,V,S)

4) Implemented a function that returns $\mathrm{Img}_r$ given $U_r$, $V_r$, and $[\sigma_0,\sigma_1,\cdots,\sigma_{r-1}]$.

In [5]:
# ---------------------------------
# Function: rebuild_img
# ---------------------------------
'''
:inputs  : U              - left singular vectors (Mxr 2D-numpy array)
:inputs  : V              - right singular vectors (Mxr 2D-numpy array)
:inputs  : S              - singular values (numpy array of size r)
:returns : imgr           - reconstructed image (MxN numpy array)
'''

def rebuild_img (U,V,S):

    imgr=U@np.diag(S)@(V.T)

    return (imgr)

5) Implemented a function that returns the normalized 2-norm of the reconstrution error $||\mathrm{Img}-\mathrm{Img}_r||/||\mathrm{Img}||$.

In [6]:
# ---------------------------------
# Function: get_compression_error
# ---------------------------------
'''
:inputs  : img            - Input grayscale image (MxN numpy array)
:inputs  : rank           - rank approxiamtion value (integer)
:returns : error          - threshold (float)
'''

def get_compression_error (img,rank):

    U, Sigma , V_T = np.linalg.svd(img)

    U=U[:,:rank]

    V_T=V_T[:rank,:]

    Sigma=Sigma[:rank]

    imgr=U@np.diag(Sigma)@V_T

    img1=np.linalg.norm(img)

    error=(np.linalg.norm(img-imgr))/img1

    return (error)

In [9]:
# # Get image from url and convert it to grayscale
url='https://images.photowall.com/products/65869/bunny-rabbit.jpg?h=699&q=85'
img = Image.open(BytesIO(requests.get(url).content)).convert('L')
img.show()

# # Convert img from a PIL object to a numpy array
img = np.asarray(img)
print(img.shape)

Threshold = 0.01
r=get_threshold_rank(img,Threshold)
print("Rank of approx.:",   r)
print("Compression ratio:", get_compression_ratio(img,r))
print("Error :",            get_compression_error(img,r))

# # Display compressed image
Ur,Vr,Sr = compress_img(img,r)
imgr = rebuild_img(Ur,Vr,Sr)
imgr = Image.fromarray(imgr)
imgr.show()

(699, 491)
Rank of approx.: 15
Compression ratio: 0.052052830782409555
Error : 0.0426569175610521
